Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [1]:
NAME = "AVISHA ANILKUMAR BHIRYANI"

---

# Latent Semantic Indexing

First we will implement latent semantic indexing (LSI)

In [2]:
### Load the input data - do not modify
import json, gzip, numpy as np
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

In [3]:
### Vectorize the text - do not modify
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vect = TfidfVectorizer(stop_words="english", sublinear_tf=True, smooth_idf=False, min_df=5)
vect.fit(texts)
vect.idf_ -= 1
idf = vect.idf_
tfidf = vect.transform(texts)
vocabulary = vect.get_feature_names_out()

## Implement LSI

Implement Latent Semantic Indexing. Do **not** use regular SVD, but instead use truncated SVD from sklearn. (Do not attempt to implement Truncated SVD yourself, use the library here.) Return weights how well the factors explain the topics.

In [22]:
# Implement LSI here
def lsi(tfidf, k):
    """Latent Semantic Indexing. Return the factors, document assignment, and factor weights"""
    from sklearn.decomposition import TruncatedSVD
    # YOUR CODE HERE
    svd = TruncatedSVD(n_components=k)
    svd.fit(tfidf)
    #V matrix - column wise
    factors = svd.components_
    #U matrix - row vise
    assignment = svd.transform(tfidf)
    weights = svd.explained_variance_
    return factors, assignment, weights

In [28]:
_tmp = lsi(tfidf, 2)
print(_tmp[0].shape)
print(_tmp[1].shape)

(2, 14153)
(10126, 2)


In [23]:
### Automatic tests. You do not need to understand or modify this code.
_tmp = lsi(tfidf, 2)
assert len(_tmp) == 3, "Incomplete result"
assert _tmp[0].shape == (2, tfidf.shape[1]), "Factor shape is not correct."
assert _tmp[1].shape == (tfidf.shape[0], 2), "Assignment shape is not correct."
del _tmp

## Explore your result

Explore the result: write a function to determine the most important words for each factor, and the most relevant documents.

In [122]:
def most_important(vocabulary, factor, k=10):
    """Most important words for each factor"""
    # YOUR CODE HERE
    dict_words = {}
    i = 0
    for key in vocabulary:
        dict_words[key] = factor[i]
        i = i+1
    sorted_words = dict(sorted(dict_words.items(), key=lambda item: item[1], reverse=True))
    arr = [each for each in list(sorted_words)[:k]]
    return arr
        
def most_relevant(assignment, k=5):
    """Most relevant documents for each factor (return document indexes)"""
    # YOUR CODE HERE
    arr = []
    dict_docs = {}
    for i in range(0,len(assignment)):
        dict_docs[i] = assignment[i]
        i = i+1
    sorted_docs = dict(sorted(dict_docs.items(), key=lambda item: item[1], reverse=True))
    arr= [each for each in list(sorted_docs)[:k]]
    return arr

def explain(vocabulary, titles, classes, factors, assignment, weights=None):
    """Print an explanation for each factor.
       If weights is None, use the relative share of the assignment weights.
       Print the ARI when assigning each document to its maximum only."""
    from sklearn.metrics import adjusted_rand_score
    # YOUR CODE HERE
    if weights is not None:
        total_factors = factors.shape[0]
        for i in range(0,total_factors):
            important_words = most_important(vocabulary, factors[i])
            important_docs = most_relevant(assignment[:i])
            print(i)
            print(important_docs)
            print(important_words)
        

In [123]:
explain(vocabulary, titles, classes, *_tmp)

0
[]
['couch', 'simpson', 'homer', 'bart', 'lisa', 'marge', 'gag', 'family', 'song', 'maggie']
1
[0]
['couch', 'gag', 'simpson', 'plot', 'maggie', 'appearances', 'character', 'season', 'family', 'sit']


In [124]:
### Automatic tests. You do not need to understand or modify this code.
_tmp = lsi(tfidf, 2)
assert len(most_important(vocabulary, _tmp[0][0], 42)) == 42, "Wrong number of most important words"
for x in most_important(vocabulary, _tmp[0][0]): assert isinstance(x, str), "Most important words are not words"
assert len(most_relevant(_tmp[1][:,0], 42)) == 42, "Wrong number of relevant results."
from unittest.mock import patch
with patch('__main__.most_important') as mock_1, patch('__main__.most_relevant') as mock_2, patch('__main__.print') as mock_3:
    explain(vocabulary, titles, classes, *_tmp)
    assert mock_1.called, "You did not use most_important"
    assert mock_2.called, "You did not use most_central"
    assert mock_3.called, "You did not print"

In [125]:
# Explore your result. These should mostly be meaningful topics!
lsi_factors, lsi_assignment, lsi_weights = lsi(tfidf, 6)
explain(vocabulary, titles, classes, lsi_factors, lsi_assignment, lsi_weights)

0
[]
['couch', 'simpson', 'homer', 'bart', 'lisa', 'marge', 'gag', 'family', 'song', 'maggie']
1
[0]
['couch', 'gag', 'simpson', 'plot', 'maggie', 'appearances', 'character', 'season', 'family', 'sit']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [126]:
### This cell contains additional tests. You do not need to modify this cell.